# Question Answering

**Delving into Extractive Question Answering**

Question answering (QA) encompasses a variety of approaches, but in this section, we'll delve into the realm of extractive question answering. This involves formulating questions about a document and pinpointing the answers as text segments within the document itself.

In essence, extractive QA resembles using a highlighter to mark relevant portions of the text and then extracting the essential information. This approach is particularly well-suited for scenarios where the answer is explicitly stated in the text.


We will refine a BERT model using the Stanford Question Answering Dataset (SQuAD) dataset, which comprises questions posed by human annotators on a collection of Wikipedia articles. This process will equip our model with the ability to generate predictions like the following:

![](images/qa1.png)

The provided example demonstrates the actual model trained and uploaded to the Hub using the code presented in this section. You can access the model and validate its predictions [here](https://huggingface.co/huggingface-course/bert-finetuned-squad?context=%F0%9F%A4%97+Transformers+is+backed+by+the+three+most+popular+deep+learning+libraries+%E2%80%94+Jax%2C+PyTorch+and+TensorFlow+%E2%80%94+with+a+seamless+integration+between+them.+It%27s+straightforward+to+train+your+models+with+one+before+loading+them+for+inference+with+the+other.&question=Which+deep+learning+libraries+back+%F0%9F%A4%97+Transformers%3F).

Encoder-only models like BERT excel at extracting answers to factual questions, such as "Who invented the Transformer architecture?" However, they struggle with open-ended questions like "Why is the sky blue?" To address these more challenging scenarios, encoder-decoder models like T5 and BART are employed. These models synthesize information in a manner akin to text summarization, generating comprehensive responses that go beyond simply extracting snippets from the provided text.

### Preparing the data

The [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) dataset serves as the primary academic benchmark for extractive question answering, and it's the dataset we'll utilize in this section. A more challenging variant, [SQuAD v2](https://huggingface.co/datasets/squad_v2), incorporates questions that lack answers. As long as your custom dataset adheres to the same structure, featuring columns for contexts, questions, and answers, you can readily adapt the steps outlined below.

### Brief Note on SQuAD Dataset
The Stanford Question Answering Dataset (SQuAD) is a popular benchmark dataset in natural language processing (NLP). It's designed for evaluating machine comprehension by requiring models to read a passage of text and answer questions about it. SQuAD contains over 100,000 question-answer pairs created by crowd workers based on articles from various sources like Wikipedia.

Each question in SQuAD is associated with a specific passage or context paragraph, and the goal is for models to generate precise answers based solely on the information in that passage. This dataset has been pivotal in the development and evaluation of question-answering systems and has led to significant advancements in NLP research, especially in the field of machine reading comprehension.

Researchers use SQuAD to train, test, and benchmark different models, aiming to achieve better accuracy and understanding in reading and answering questions. It's been instrumental in the development of various models like BERT, RoBERTa, and other transformer-based architectures in NLP.

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### The SQuAD dataset

With the convenient `load_dataset()` function, we can effortlessly download and store the dataset in a single step.

![](images/qa2.png)

The dataset appears to contain all the necessary information, with fields for `context`, `question`, and `answers`. Let's examine the first element of our training set to confirm:

![](images/qa3.png)

The `context` and `question` fields are straightforward to interpret and utilize. The `answers` field, however, requires a bit more attention as it's a dictionary containing two lists. This specific format is expected by the `squad` metric during evaluation. If you're working with your own dataset, you don't need to adhere to this exact structure. The `text` field is self-explanatory, while the `answer_start` field indicates the starting character index of each answer within the context.

Since the training process involves a single valid answer for each question, we can verify this claim by employing the `Dataset.filter()` method. This method allows us to filter the dataset based on specific criteria, ensuring that only instances with a single answer remain:

![](images/qa4.png)

During evaluation, each sample may have multiple valid answers, which can be identical or distinct from each other. This is in contrast to the training phase, where each question has a single correct answer.

![](images/qa5.png)

We'll refrain from delving into the evaluation script itself, as a 🤗 Datasets metric will handle this aspect for us. However, in essence, some questions have multiple acceptable answers, and the script compares a predicted answer to all the valid options, selecting the one with the highest score. For example, examining the sample at index 2:

![](images/qa6.png)

Indeed, we can observe that the answer can be one of the three possibilities we encountered earlier. This highlights the presence of multiple valid answers for certain questions, which the evaluation script will account for during assessment.

### Processing the training data

Before diving into label generation, let's tackle the first step: preprocessing the training data. This involves converting the raw text into a format that the model can understand, which requires the utilization of a tokenizer. The crux of the matter lies in generating labels for the question's answer, which entails determining the start and end positions of the answer's corresponding tokens within the context. However, let's not rush ahead. First, let's focus on the initial step of text tokenization.

![](images/qa7.png)

As mentioned earlier, we'll be fine-tuning a BERT model for this task, but you can employ any other model architecture as long as it provides a fast tokenizer implementation. You can refer to this [comprehensive table](https://huggingface.co/transformers/#supported-frameworks) to explore the various architectures that come with a fast tokenizer version. To verify whether the `tokenizer` object you're using is indeed backed by 🤗 Tokenizers, simply check its `is_fast` attribute.

![](images/qa8.png)

We can feed both the question and the context to our tokenizer, and it will seamlessly incorporate the special tokens, resulting in a sentence structured like this:

```
[CLS] question [SEP] context [SEP]
```
This format ensures that the model can effectively process the question in relation to the context, facilitating the task of identifying the answer's location within the context.

In the context of natural language processing and specifically regarding the BERT (Bidirectional Encoder Representations from Transformers) model architecture, "CLS" stands for "Classification" and "SEP" stands for "Separator."

- **CLS** (Classification): In BERT, the "[CLS]" token is added to the input text at the beginning of a sequence. It's used to represent the entire input sequence when the model is fine-tuned for classification tasks. The output vector corresponding to this token is used for classification purposes, such as text categorization or sentence pair classification.

- **SEP** (Separator): The "[SEP]" token is used to separate pairs of sentences in BERT. It is placed between two sentences in a pair or between segments of a single sentence when the model requires two separate sequences of text, indicating to the model the boundary between different segments. It helps BERT understand and distinguish between different sentences or segments during training and inference.

These tokens play crucial roles in how BERT processes and understands input sequences, particularly in tasks involving classification or sentence-level understanding.

Let’s double-check:

![](images/q9.png)

The labels will represent the indices of the tokens that mark the start and end of the answer. The model's task is to predict a start and end logit for each token in the input sequence. The theoretical labels would be as follows:

![](images/qa10.png)

While the context in this example is relatively short, some instances within the dataset contain lengthy contexts that would surpass the maximum length we've established (384 tokens in this case). 


To handle lengthy contexts, we will create multiple training features from a single sample in our dataset by employing a sliding window approach. This technique involves dividing the context into smaller segments while maintaining the essential information. To illustrate this process using the current example, we'll restrict the maximum length to 100 tokens and utilize a sliding window of 50 tokens. As a reminder, we employ the following parameters:

* `max_length`: Sets the maximum length of the input sequence (here, 100 tokens)

* `truncation="only_second"`: Truncates the context (which occupies the second position) when the combined length of the question and context exceeds the maximum length

* `stride`: Defines the number of overlapping tokens between two consecutive chunks (here, 50 tokens)

* `return_overflowing_tokens=True`: Informs the tokenizer to retain the overflowing tokens for subsequent processing

This approach effectively breaks down the lengthy context into manageable segments, allowing the model to process each segment independently while maintaining the context's overall meaning.

![](images/qa11.png)

As we can observe, our example has been divided into four distinct inputs, each comprising the question and a portion of the context. It's important to note that the answer to the question ("Bernadette Soubirous") only appears in the third and final inputs. By addressing lengthy contexts in this manner, we inadvertently generate training examples where the answer is absent from the context. For these instances, the labels will be `start_position = end_position = 0`, indicating that we should predict the `[CLS]` token. Additionally, we'll assign these labels in the unfortunate circumstance where the answer has been truncated, leaving us with only its beginning (or end). For examples where the answer is entirely contained within the context, the labels will represent the index of the token where the answer begins and the index of the token where it ends. This approach effectively handles situations where the answer is absent or truncated, ensuring that the model is trained to handle such scenarios.

The dataset conveniently provides us with the starting character of the answer within the context. By adding the length of the answer, we can determine the ending character in the context.  To retrieve these mappings, we can pass the return_offsets_mapping=True parameter to the tokenizer:

![](images/qa12.png)

The returned results include the familiar input IDs, token type IDs, and attention mask, along with the desired offset mapping and an additional key named `overflow_to_sample_mapping`. The corresponding value will prove useful when tokenizing multiple texts simultaneously, which we should leverage to harness the efficiency of our Rust-backed tokenizer. Since a single sample can generate multiple features, this mapping associates each feature with its corresponding source example. As we only tokenized one example in this instance, we obtain a list of `0s`.

![](images/qa13.png)

But if we tokenize more examples, this will become more useful:

![](images/qa14.png)

As we can observe, the first three examples (indices 2, 3, and 4 in the training set) each generated four features, while the last example (index 5 in the training set) produced seven features. This information will be crucial in associating each feature with its corresponding label. As previously mentioned, the labels are as follows:

- (0, 0): Indicates that the answer is absent from the corresponding context segment
- (start_position, end_position): Signifies that the answer is present within the corresponding context segment, where `start_position` represents the index of the token (in the input IDs) marking the start of the answer and `end_position` represents the index of the token (in the input IDs) marking the end of the answer

This distinction between labels for cases where the answer is present or absent within the context segment is crucial for accurate training of the model.

To determine whether the answer is present or absent within the corresponding context segment and, if applicable, identify the positions of the answer's tokens, we begin by locating the indices marking the start and end of the context within the input IDs. While token type IDs could be used for this purpose, not all models utilize them (DistilBERT, for instance, does not require them). Therefore, we'll employ the `sequence_ids()` method of the `BatchEncoding` object returned by our tokenizer.

Once these token indices are obtained, we examine the corresponding offsets, which are tuples containing two integers representing the character span within the original context. This allows us to determine if the context segment in this feature begins after the answer or ends before the answer begins (in which case the label is `(0, 0)`). If this is not the case, we iterate to identify the first and last tokens of the answer:

![](images/qa15.png)

To validate the accuracy of our approach, let's examine a few results. For the first feature, we identify the labels `(83, 85)`. Let's compare the theoretical answer with the decoded span of tokens from 83 to 85 (inclusive):

![](images/qa16.png)

Indeed, the decoded span precisely matches the theoretical answer, effectively validating our approach in identifying the correct tokens corresponding to the answer. Now, let's examine index 4, where we intentionally assigned the labels `(0, 0)`, indicating that the answer is absent from the context chunk of that feature.

![](images/qa17.png)

Indeed, we don’t see the answer inside the context.

Having thoroughly explored the step-by-step process of preprocessing our training data, we can now encapsulate these steps into a function that can be applied to the entire training dataset. Since most contexts are lengthy and will be divided into multiple features, we'll employ static padding to align all features to the predetermined maximum length. This approach is more efficient than dynamic padding in this scenario, as the majority of contexts will indeed exceed the maximum length.

![](images/qa18.png)

Note that we defined two constants to specify the maximum length and the length of the sliding window. Additionally, we implemented a minor cleanup step before tokenization: certain questions in the SQuAD dataset contain unnecessary leading and trailing spaces that do not contribute to the meaning (and consume additional space during tokenization when using a model like RoBERTa). These extra spaces were removed during this cleanup phase.

To apply this function to the entire training set, we utilize the `Dataset.map()` method with the `batched=True` flag. This is essential as we are altering the length of the dataset (since a single example can generate multiple training features). Here's an example of how to apply the function:

![](images/qa19.png)

The preprocessing step successfully introduced approximately 1,000 additional features, expanding our training set. Now, with the training set prepared, let's delve into the preprocessing of the validation set!

### Processing the validation data

Preprocessing the validation data will be a less complex task as we don't need to generate labels (unless we intend to calculate a validation loss, but this metric won't provide a comprehensive understanding of the model's performance). The true challenge lies in interpreting the model's predictions and converting them into meaningful spans within the original context. To achieve this, we simply need to retain both the offset mappings and a mechanism for associating each generated feature with its corresponding original example. Since the original dataset includes an ID column, we'll utilize this identifier for this purpose.

The only additional step we'll introduce here is a minor cleanup of the offset mappings. These mappings will include offsets for both the question and the context. However, once we reach the post-processing stage, we won't have a means of distinguishing which portion of the input IDs corresponds to the context and which belongs to the question (the `sequence_ids()` method we employed is only available for the tokenizer's output). Therefore, we'll set the offsets associated with the question to `None`.

This straightforward preprocessing step effectively prepares the validation data for subsequent analysis and evaluation of the model's predictions.

![](images/qa20.png)

We can apply this function on the whole validation dataset like before:

![](images/qa21.png)

In this instance, we've only introduced a few hundred additional samples, suggesting that the contexts in the validation dataset are relatively shorter.

With all the data now preprocessed, we're ready to embark on the training phase.

### Fine-tuning the model with the Trainer API


The training code for this example will closely resemble the code from previous sections, with the primary challenge being the implementation of the `compute_metrics()` function. Since we padded all samples to the maximum length, a data collator is not required, making the metric computation the primary focus. The complexity lies in post-processing the model's predictions into spans of text within the original examples. Once this is accomplished, the metric provided by the 🤗 Datasets library will handle most of the evaluation tasks.

#### Post-processing

The model will generate logits for the start and end positions of the answer within the input IDs. The post-processing stage will follow a similar approach to the one we employed there, so here's a brief recap of the steps involved:

1. **Masking Irrelevant Tokens:** We'll mask the start and end logits corresponding to tokens that fall outside the context. This ensures that the model's attention is focused solely on the relevant portions of the input sequence.

2. **Probabilistic Conversion:** We'll convert the start and end logits into probabilities using a softmax function. This transformation effectively scales the logits into a range between 0 and 1, representing the likelihood of each token being the start or end of the answer.

3. **Scoring Answer Spans:** We'll assign a score to each pair of `(start_token, end_token)` by multiplying the corresponding start and end probabilities. This scoring mechanism prioritizes answer spans with higher confidence levels.

4. **Identifying the Optimal Answer Span:** We'll identify the pair of `(start_token, end_token)` with the highest score, ensuring that the `start_token` precedes the `end_token`. This pair represents the most likely and valid answer span.

By following these steps, we can effectively extract the most probable answer span from the model's predictions, enabling us to interpret and evaluate the model's performance.

In this instance, we'll introduce slight modifications to the process since we only need to identify the predicted answer and not calculate actual scores. Consequently, we can bypass the softmax step, streamlining the process. To further enhance speed, we'll limit our analysis to the top `n_best` logits, eliminating the need to evaluate all possible `(start_token, end_token)` pairs. With `n_best=20`, we'll focus on the 20 most promising logits. Since we're not using softmax, the scores will be represented as logit scores, obtained by summing the start and end logits (rather than multiplying them, as the rule `log(ab) = log(a) + log(b)` states).

To illustrate this approach, we'll utilize predictions generated by the default QA pipeline model on a small subset of the validation set. We can employ the same processing function as before; as it relies on the global constant `tokenizer`, we simply need to replace this object with the tokenizer of the model we intend to use temporarily.

This modified approach will enable us to efficiently extract the predicted answer spans from the model's predictions, facilitating the evaluation of the model's performance.

![](images/qa22.png)

Next, we'll remove any unnecessary columns from our `eval_set` that the model does not require. Subsequently, we'll create a batch encompassing the entire small validation set and pass it through the model for processing. If a GPU is available, we'll utilize it to accelerate the computation.

This step involves preparing the validation data for model evaluation by removing irrelevant columns and constructing a batch containing the entire validation set. Leveraging a GPU, if available, will expedite the processing of the validation data.

![](images/qa23.png)

To identify the predicted answer for each example in our `small_eval_set`, we need to account for the possibility that a single example may have been divided into multiple features during the preprocessing stage. Therefore, the initial step involves establishing a mapping between each example in `small_eval_set` and its corresponding features in `eval_set`. This mapping ensures that we associate the predicted answer with the correct example.

![](images/qa24.png)

Armed with this mapping, we can now proceed with identifying the predicted answer for each example in `small_eval_set`. This involves iterating through all the examples and, for each example, examining all the associated features. As mentioned earlier, we'll focus on the logit scores for the top `n_best` start logits and end logits, excluding positions that result in:

1. An answer that falls outside the context
2. An answer with a negative length
3. An answer that exceeds the maximum acceptable length (set as `max_answer_length=30`)

Once we've compiled all the scored possible answers for a given example, we simply select the one with the highest logit score. This selected answer represents the model's prediction for that particular example.

![](images/qa25.png)

Finally, we need to ensure that the predicted answers are in the format expected by the metric we intend to use. As is customary, we'll utilize the 🤗 Evaluate library to load the metric and evaluate the model's performance. This step involves formatting the predicted answers to align with the metric's input requirements, ensuring compatibility and accurate evaluation.

![](images/qa26.png)

The chosen metric anticipates the predicted answers in the format we encountered earlier (a list of dictionaries containing one key for the example ID and another key for the predicted text). Similarly, it expects the ground-truth answers in the format described below (a list of dictionaries containing one key for the example ID and another key for the possible answers):

![](images/qa27.png)

The achieved results are quite satisfactory, especially considering that DistilBERT fine-tuned on SQuAD reportedly achieves scores of 79.1 and 86.9 on the entire dataset.

Now, let's consolidate the aforementioned steps into a `compute_metrics()` function, which will be utilized within the `Trainer`. Typically, the `compute_metrics()` function receives a tuple `eval_preds` containing logits and labels. However, in our case, we require additional information as we need to access the offset mapping from the features dataset and the original contexts from the examples dataset. Consequently, this function cannot be employed to obtain regular evaluation results during training. Instead, it will be exclusively used for evaluating the model's performance at the end of the training process.

The `compute_metrics()` function encapsulates the same steps as before, with the addition of a minor check to handle situations where no valid answers are identified. In such cases, an empty string will be predicted.

![](images/qa28.png)

We can check it works on our predictions:

![](images/qa29.png)

Looking good! Now let’s use this to fine-tune our model.

### Fine-tuning the model

We are now ready to train our model. Let’s create it first, using the AutoModelForQuestionAnswering class like before:

![](images/qa30.png)

As is common with fine-tuning pre-trained models, the warning message indicates that certain weights from the pre-training head are not utilized, while others associated with the question answering head are randomly initialized. This implies that the model is not yet ready for immediate use and requires fine-tuning. Fortunately, we are about to embark on the fine-tuning process to address this issue.

Let’s take a look at our TrainingArguments:

![](images/qa31.png)

We’ve seen most of these before: we set some hyperparameters (like the learning rate, the number of epochs we train for, and some weight decay)
Finally, we just pass everything to the Trainer class and launch the training:

![](images/qa32.png)

Since the complete training process can be time-consuming (approximately an hour on a Titan RTX GPU), feel free to take a break during this interval. You can use this time to enjoy a cup of coffee or revisit some of the more challenging aspects of this course.

Upon completion of the training process, we can finally assess the performance of our model (hoping that our computational efforts were not in vain). The `predict()` method of the `Trainer` class will return a tuple where the initial elements represent the model's predictions (in this case, a pair comprising the start and end logits). We'll pass these predictions to our `compute_metrics()` function:

![](images/qa33.png)

Great! As a comparison, the baseline scores reported in the BERT article for this model are 80.8 and 88.5, so we’re right where we should be.

### A custom training loop

The provided code snippet outlines the complete training loop for the text summarization model, offering flexibility for customization as needed. It closely resembles the training loop presented in Chapter 3, with the exception of the evaluation loop. Due to the absence of the `Trainer` class, we can perform regular model evaluation.

#### Preparing everything for training

The initial step involves constructing `DataLoaders` from the respective datasets. We specify the dataset format as `"torch"` and eliminate the unused columns from the validation set. Next, we utilize the `default_data_collator` provided by Transformers as the `collate_fn` and shuffle the training set while maintaining the order of the validation set.

![](images/qa34.png)


To ensure a fresh start without carrying over any previous fine-tuning, we reinitialize the model. This reinstantiation ensures that the model begins training from the pre-trained BERT model's initial parameters.

![](images/qa35.png)

Then we will need an optimizer. As usual we use the classic AdamW, which is like Adam, but with a fix in the way weight decay is applied:

![](images/qa37.png)

Once all the necessary objects are created, we invoke the `accelerator.prepare()` method to prepare the environment for training. If you intend to train on TPUs within a Colab notebook, it's essential to encapsulate this code within a training function and refrain from executing any cell that instantiates an `Accelerator`. To activate mixed-precision training, pass `fp16=True` to the `Accelerator` constructor (or, when executing the code as a script, ensure that the 🤗 Accelerate configuration is set accordingly).

![](images/qa36.png)

Unlike before, we can't directly determine the number of training steps from the `train_dataloader` length after it's been through the `accelerator.prepare()` method. This is due to potential modifications made by the accelerator, such as gradient accumulation. Therefore, we rely on the `train_dataloader` length returned by the `accelerator` itself to calculate the number of steps for our linear learning rate schedule. Similar to previous sections, we'll utilize the same linear schedule for adjusting the learning rate throughout the training process:

![](images/qa38.png)

To publish our model on the Hugging Face Hub, we'll need to create a `Repository` object within a working directory. Before proceeding, ensure you're logged in to the Hugging Face Hub. The repository name will be derived from the desired model ID (feel free to modify the `repo_name` with your preferred choice; it must include your username, which is extracted by the `get_full_repo_name()` function).

![](images/qa39.png)

Then we can clone that repository in a local folder. If it already exists, this local folder should be a clone of the repository we are working with:

![](images/qa40.png)

To upload the intermediate models to the Hub, we can utilize the `repo.push_to_hub()` method, invoking it with the path to the `output_dir`. This approach enables us to upload the intermediate models at the conclusion of each epoch.

#### Training Loop Breakdown

Now, let's dive into the heart of the code: the complete training loop! This loop can be divided into three main stages:

**1. Training:**

* We embark on a classic training journey, iterating over the `train_dataloader`.
* Each iteration involves feeding the data through the model's forward pass.
* Then, we compute the error and perform a backward pass to update the model's parameters.
* Finally, we take a step towards minimizing the error with the optimizer.

**2. Evaluation:**

* After each training epoch, we take a break to assess the model's performance.
* We gather all the predictions for the start and end logits across the entire validation set.
* These predictions are converted to NumPy arrays for further analysis.
* Once we've collected all the evaluation results, we combine them into a single, consolidated set.
* **Note:** We need to be mindful of potential padding added by the `Accelerator` to ensure equal sample sizes across processes. Therefore, we perform a slight truncation to remove any excess padding.

**3. Saving and Uploading:**

* We take a snapshot of the model's progress by saving the current model and tokenizer.
* Next, we send the saved model and tokenizer to the Hugging Face Hub using the `repo.push_to_hub()` method.
* To avoid slowing down the training, we utilize the `blocking=False` argument, allowing the upload to occur asynchronously in the background. This ensures the training process continues uninterrupted while the saved model finds its new home on the Hub.

**Complete Code:**

The provided code snippet captures the entire training loop in its entirety. Each stage is meticulously outlined, providing a clear roadmap for training and evaluating the bilingual summarization model.

![](images/qa41.png)

### Using the fine-tuned model

If you prefer your bilingual summarization model on your own machine, you can leverage the power of pipelines! Simply specifying the model identifier unlocks its potential within your local environment. No additional configuration required – just a convenient identifier and you're ready to roll!

![](images/qa42.png)

Great! Our model is working as well as the default one for this pipeline!